<a href="https://colab.research.google.com/github/ManahilKashif/AI201/blob/main/project_no_03_functioncalling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain_community langchain_google_genai

In [2]:
from google.colab import userdata
import os
os.environ['GOOGLE_API_KEY']=userdata.get('GOOGLE_API_KEY')

In [3]:


from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash',verbose=True)

llm.invoke("Hi")


AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-f6ababc0-b5c6-4dcf-9d23-27f0106ff30d-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})

In [4]:

from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

llm_with_tools = llm.bind_tools([multiply])

In [5]:

tools_list=[multiply]
model_with_tools = llm.bind_tools(tools_list)

In [6]:

result = llm_with_tools.invoke("Hey how are you!")
result

AIMessage(content="I'm doing well, thank you for asking! How are you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-9934ea2f-9a53-4223-916e-dabd38177436-0', usage_metadata={'input_tokens': 68, 'output_tokens': 17, 'total_tokens': 85, 'input_token_details': {'cache_read': 0}})

In [7]:
result = llm_with_tools.invoke("What is 2 multiplied by 3?")
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 3.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-4b35a3fc-7b56-460b-8e45-da3f0df9d9b3-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2.0, 'b': 3.0}, 'id': '8f7597dd-6e6b-4cba-94b7-4cfd71397769', 'type': 'tool_call'}], usage_metadata={'input_tokens': 72, 'output_tokens': 3, 'total_tokens': 75, 'input_token_details': {'cache_read': 0}})

In [15]:
from langchain.tools import Tool

# Example tool for addition
def divide_numbers(a, b):
    return a / b

math_tool = Tool(
    name="AdditionTool",
    func=lambda x: f"The result is: {divide_numbers(*map(int, x.split()))}",
    description="Adds two numbers. Input should be two integers separated by a space."
)

In [16]:

from langchain.agents import initialize_agent, AgentType

# List of tools
tools = [math_tool]

# Initialize the agent
agent = initialize_agent(
    tools=tools,                        # Provide the tools
    llm=llm,                            # LLM for fallback
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Agent type
    verbose=True                        # Enable debugging output
)

In [17]:
# Query the agent
response1 = agent.run("What is 100 / 20?")  # Uses the tool
print(response1)



> Entering new AgentExecutor chain...
Thought: I can solve this by finding a number that when multiplied by 20 equals 100.  Alternatively, I can use repeated subtraction.  Let's use a method that leverages the available tool.  I can repeatedly subtract 20 from 100 until I reach 0, counting the number of subtractions. This is equivalent to division.


Action: AdditionTool
Action Input: 100 -20
Observation: The result is: -5.0
Thought:Question: What is 100 / 20?
Thought: I can solve this by repeatedly subtracting 20 from 100 until I reach 0, counting the number of subtractions.  The AdditionTool isn't directly suited for subtraction, but I can use it to add negative numbers.  Let's try a different approach. I can use the AdditionTool to find what number multiplied by 20 equals 100.  I'll try different multiples of 20.

Action: AdditionTool
Action Input: 20 20
Observation: The result is: 1.0
Thought:Question: What is 100 / 20?
Thought: I can solve this by finding a number that when mult